In [7]:
import sys
sys.path.append("../src")
sys.path.append("../data")

from leitura_dados import ler_arquivo
from estatisticas import *

grafo = ler_arquivo("../data/mggdb_0.25_4.dat")

print("\n--- Totais ---")
print(f"Quantidade Total de Vértices: {quantidade_vertices(grafo)}")
print(f"Quantidade Total de Arestas: {quantidade_arestas(grafo)}")
print(f"Quantidade Total de Arcos: {quantidade_arcos(grafo)}")

print("\n--- Requeridos ---")
print(f"Quantidade de Vértices Requeridos: {quantidade_vertices_requeridos(grafo)}")
print(f"Quantidade de Arestas Requeridas: {quantidade_arestas_requeridas(grafo)}")
print(f"Quantidade de Arcos Requeridos: {quantidade_arcos_requeridos(grafo)}")

print("\n--- Outros ---")
print("Densidade do Grafo:", densidade(grafo))
print("Componentes Conectados:", componentes_conectados(grafo))
print("Grau (mínimo, máximo):", grau_min_max(grafo))

caminhos, caminhos_completos = floyd_warshall(grafo)

print("Intermediação:", intermediacao(grafo, caminhos_completos)) 
print("Caminho Médio:", caminho_medio(caminhos))
print("Diâmetro:", diametro(caminhos))


--- Totais ---
Quantidade Total de Vértices: 15
Quantidade Total de Arestas: 4
Quantidade Total de Arcos: 30

--- Requeridos ---
Quantidade de Vértices Requeridos: 4
Quantidade de Arestas Requeridas: 3
Quantidade de Arcos Requeridos: 11

--- Outros ---
Densidade do Grafo: 0.1619047619047619
Componentes Conectados: 1
Grau (mínimo, máximo): (0, 5)
Intermediação: {'N3': 0, 'N7': 0, 'N8': 0, 'N9': 0, '1': 12, '10': 14, '2': 20, '4': 0, '3': 6, '7': 0, '8': 8, '5': 12, '11': 6, '9': 18, '6': 8}
Caminho Médio: 19.89090909090909
Diâmetro: 36
